In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()

client = OpenAI(api_key=os.environ['API_KEY'])

MODEL_NAME = "gpt-4o-mini"

In [6]:
df = pd.read_excel('sample01.xlsx', sheet_name='Sheet1')
df.head()

,Date,Product,Region,Quantity,UnitPrice,Sales
0,2024-04-12,Tablet,North,5,870,4350
1,2024-06-28,PC,East,4,536,2144
2,2024-04-02,Accessory,South,8,122,976
3,2024-01-15,Tablet,North,7,664,4648
4,2024-04-16,Accessory,South,2,776,1552


In [15]:
data_text = df.astype(str)
type(df)

pandas.core.frame.DataFrame

In [ ]:
df["Month"] = df["Date"].dt.to_period("M").dt.to_timestamp()
monthly_sales = df.groupby("Month", as_index=False)["Sales"].sum()
df["Day"] = df["Date"].dt.to_period("D").dt.to_timestamp()
daily_sales = df.groupby("Day", as_index=False)["Sales"].sum()
df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()
weekly_sales = df.groupby("Week", as_index=False)["Sales"].sum()
avg_unitprice_by_product = df.groupby("Product", as_index=False)["UnitPrice"].mean()
avg_unitprice_by_product

,Product,UnitPrice
0,Accessory,424.750000
1,PC,536.000000
2,Phone,183.000000
3,Tablet,782.333333


In [ ]:
prompt_text = f"売上データ:\n{data_text}\nこのデータをもとに、(1)月ごとの売上合計を計算, (2)商品別の平均単価, (3)地域ごとに最も売れた商品をそれぞれ算出し、(4)全体の傾向を分析してください。\n(1)〜(4)をJson形式で返してください。